## Modeling Receptions

This notebook builds out a model to predict receptions.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.linear_model import LinearRegression, Ridge, RidgeCV, Lasso, LassoCV, ElasticNet, ElasticNetCV
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor 
from sklearn.ensemble import GradientBoostingRegressor, AdaBoostRegressor, BaggingRegressor, StackingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn import metrics

np.random.seed(7)

In [2]:
df = pd.read_csv('../data/receiving_train.csv', index_col = 'Player')
df.dropna(inplace = True)
df.head()

,Tm,Age,G,GS,Tgt,Rec,Yds,TD,1D,YBC,...,Yds_-2_year,TD_-2_year,Rnd,Pick,Pos,YrsPlayed,Tgt_target,Rec_target,Yds_target,TD_target
Player,,,,,,,,,,,,,,,,,,,,,
Stefon Diggs,BUF,27,16,15,166,127,1535,8,73.0,1071,...,1021.0,9.0,5.0,146.0,WR,5.0,164,103,1225,10
Davante Adams,GNB,28,14,14,149,115,1374,18,73.0,777,...,1386.0,13.0,2.0,53.0,WR,6.0,169,123,1553,11
DeAndre Hopkins,ARI,28,16,16,160,115,1407,6,75.0,873,...,1572.0,11.0,1.0,27.0,WR,7.0,64,42,572,8
Darren Waller,LVR,28,16,15,145,107,1196,9,69.0,624,...,75.0,0.0,6.0,204.0,TE,5.0,93,55,665,2
Travis Kelce,KAN,31,15,15,145,105,1416,11,79.0,829,...,1336.0,10.0,3.0,63.0,TE,7.0,134,92,1125,9


In [3]:
df.shape

(985, 44)

### Baseline Score

In [4]:
df[['Tgt_target', 'Rec_target', 'Yds_target', 'TD_target']].mean()

Tgt_target     43.713706
Rec_target     29.422335
Yds_target    329.638579
TD_target       2.082234
dtype: float64

## Models

In [5]:
X = df.drop(columns = ['Tm', 'Pos', 'Player-additional', 'Tgt_target', 'Rec_target', 'Yds_target', 'TD_target', 'Year'])
y = df['Rec_target']

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 7)

In [7]:
lr = LinearRegression()

lr.fit(X_train, y_train)

print(f'Training Score: {lr.score(X_train, y_train)}')
print(f'Testing Score: {lr.score(X_test, y_test)}')

Training Score: 0.542150950158006
Testing Score: 0.4749748293647835


In [8]:
knn = KNeighborsRegressor()

knn.fit(X_train, y_train)

print(f'Training Score: {knn.score(X_train, y_train)}')
print(f'Testing Score: {knn.score(X_test, y_test)}')

Training Score: 0.5836736815059362
Testing Score: 0.47005136444858364


In [9]:
dt = DecisionTreeRegressor()

dt.fit(X_train, y_train)

print(f'Training Score: {dt.score(X_train, y_train)}')
print(f'Testing Score: {dt.score(X_test, y_test)}')

Training Score: 1.0
Testing Score: 0.036272534758269526


In [10]:
rf = RandomForestRegressor()

rf.fit(X_train, y_train)

print(f'Training Score: {rf.score(X_train, y_train)}')
print(f'Testing Score: {rf.score(X_test, y_test)}')

Training Score: 0.9262718455281006
Testing Score: 0.5036275979392388


In [11]:
gr = GradientBoostingRegressor()

gr.fit(X_train, y_train)

print(f'Training Score: {gr.score(X_train, y_train)}')
print(f'Testing Score: {gr.score(X_test, y_test)}')

Training Score: 0.8160349429464873
Testing Score: 0.502265047962388


In [12]:
ada = AdaBoostRegressor()

ada.fit(X_train, y_train)

print(f'Training Score: {ada.score(X_train, y_train)}')
print(f'Testing Score: {ada.score(X_test, y_test)}')

Training Score: 0.5354681352273543
Testing Score: 0.3886517858528602


## Gridsearching

I want to add run some Gridsearches to see if we can improve any of our models:

In [13]:
rf2 = RandomForestRegressor()

rf_params = {
    'n_estimators': [100, 150, 200],
    'max_depth': [None, 1, 2, 3, 4, 5]
}
gs = GridSearchCV(rf2, param_grid = rf_params, cv = 5)
gs.fit(X_train, y_train)
print(gs.best_score_)
print(gs.best_params_)

print(f'Training Score: {gs.score(X_train, y_train)}')
print(f'Testing Score: {gs.score(X_test, y_test)}')

0.46611282429594275
{'max_depth': 4, 'n_estimators': 100}
Training Score: 0.6362950416270368
Testing Score: 0.5132162920560777


In [14]:
gb2 = GradientBoostingRegressor()

gb_params = {
    'learning_rate': [.1, .05, .2],
    'n_estimators': [100, 150, 200],
    'max_depth': [None, 1, 2, 3, 4, 5]
}
gs2 = GridSearchCV(gb2, param_grid = gb_params, cv = 5)
gs2.fit(X_train, y_train)
print(gs2.best_score_)
print(gs2.best_params_)

print(f'Training Score: {gs2.score(X_train, y_train)}')
print(f'Testing Score: {gs2.score(X_test, y_test)}')

0.4447927872553333
{'learning_rate': 0.05, 'max_depth': 1, 'n_estimators': 100}
Training Score: 0.5439919300017233
Testing Score: 0.4818720972880871


In [15]:
ada2 = AdaBoostRegressor()

ada_params = {
    'learning_rate': [.1, .05, .2],
    'n_estimators': [100, 150, 200],
}
gs3 = GridSearchCV(ada2, param_grid = ada_params, cv = 5)
gs3.fit(X_train, y_train)
print(gs3.best_score_)
print(gs3.best_params_)

print(f'Training Score: {gs3.score(X_train, y_train)}')
print(f'Testing Score: {gs3.score(X_test, y_test)}')

0.4410152291934743
{'learning_rate': 0.1, 'n_estimators': 100}
Training Score: 0.5813994473296171
Testing Score: 0.47291021368102637


Our RandomForest model looks better after running the Gridsearch, as does our AdaBoosted Model. Our GradientBoosted Model actually ended up being worse than the non-Gridsearched model.

## Ridge

Next, I will try some regularization using a Ridge model.

In [16]:
X = df.drop(columns = ['Tm', 'Pos', 'Player-additional', 'Tgt_target', 'Rec_target', 'Yds_target', 'TD_target', 'Year'])
y = df['Rec_target']

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 7)

In [18]:
ss = StandardScaler()
X_train_sc = ss.fit_transform(X_train)
X_test_sc = ss.transform(X_test)

Ridge with Default alpha = 1.0

In [19]:
ridge = Ridge()

ridge.fit(X_train_sc, y_train)

print(f'Training Score: {ridge.score(X_train_sc, y_train)}')
print(f'Testing Score: {ridge.score(X_test_sc, y_test)}')

Training Score: 0.5420353523442707
Testing Score: 0.4780094653640059


Find the best alpha

In [20]:
alphas = np.logspace(0, 5, 100)
ridge_cv = RidgeCV(alphas = alphas, cv = 5)
ridge_cv.fit(X_train_sc, y_train)

RidgeCV(alphas=array([1.00000000e+00, 1.12332403e+00, 1.26185688e+00, 1.41747416e+00,
       1.59228279e+00, 1.78864953e+00, 2.00923300e+00, 2.25701972e+00,
       2.53536449e+00, 2.84803587e+00, 3.19926714e+00, 3.59381366e+00,
       4.03701726e+00, 4.53487851e+00, 5.09413801e+00, 5.72236766e+00,
       6.42807312e+00, 7.22080902e+00, 8.11130831e+00, 9.11162756e+00,
       1.02353102e+01, 1.14975700e+0...
       6.89261210e+03, 7.74263683e+03, 8.69749003e+03, 9.77009957e+03,
       1.09749877e+04, 1.23284674e+04, 1.38488637e+04, 1.55567614e+04,
       1.74752840e+04, 1.96304065e+04, 2.20513074e+04, 2.47707636e+04,
       2.78255940e+04, 3.12571585e+04, 3.51119173e+04, 3.94420606e+04,
       4.43062146e+04, 4.97702356e+04, 5.59081018e+04, 6.28029144e+04,
       7.05480231e+04, 7.92482898e+04, 8.90215085e+04, 1.00000000e+05]),
        cv=5)

In [21]:
ridge_cv.alpha_

236.4489412645407

In [22]:
ridge_cv.best_score_

0.4960950270266509

In [23]:
print(f'Training Score: {ridge_cv.score(X_train_sc, y_train)}')
print(f'Testing Score: {ridge_cv.score(X_test_sc, y_test)}')

Training Score: 0.527154056620251
Testing Score: 0.49530594317825816


## Lasso

And after the Ridge model, I will try a Lasso Model.

In [24]:
lasso = Lasso()

lasso.fit(X_train_sc, y_train)

print(f'Training Score: {lasso.score(X_train_sc, y_train)}')
print(f'Testing Score: {lasso.score(X_test_sc, y_test)}')

Training Score: 0.5193778926160126
Testing Score: 0.4880553509819028


In [25]:
l_alphas = np.logspace(-3, 0, 100)
lasso_cv = LassoCV(alphas = l_alphas, max_iter = 10000)
lasso_cv.fit(X_train_sc, y_train)

LassoCV(alphas=array([0.001     , 0.00107227, 0.00114976, 0.00123285, 0.00132194,
       0.00141747, 0.00151991, 0.00162975, 0.00174753, 0.00187382,
       0.00200923, 0.00215443, 0.00231013, 0.00247708, 0.00265609,
       0.00284804, 0.00305386, 0.00327455, 0.00351119, 0.00376494,
       0.00403702, 0.00432876, 0.00464159, 0.00497702, 0.0053367 ,
       0.00572237, 0.00613591, 0.00657933, 0.0070548 , 0.00756463,
       0.008...
       0.09326033, 0.1       , 0.10722672, 0.1149757 , 0.12328467,
       0.13219411, 0.14174742, 0.15199111, 0.16297508, 0.17475284,
       0.18738174, 0.2009233 , 0.21544347, 0.23101297, 0.24770764,
       0.26560878, 0.28480359, 0.30538555, 0.32745492, 0.35111917,
       0.37649358, 0.40370173, 0.43287613, 0.46415888, 0.49770236,
       0.53366992, 0.57223677, 0.61359073, 0.65793322, 0.70548023,
       0.75646333, 0.81113083, 0.869749  , 0.93260335, 1.        ]),
        max_iter=10000)

In [26]:
lasso_cv.alpha_

0.5336699231206312

In [27]:
print(f'Training Score: {lasso_cv.score(X_train_sc, y_train)}')
print(f'Testing Score: {lasso_cv.score(X_test_sc, y_test)}')

Training Score: 0.5284930196531243
Testing Score: 0.49019106554199476


In [28]:
lasso_coefs = lasso_cv.coef_
lasso_coefs

array([-0.        , -0.        , -0.        ,  0.        ,  5.33596555,
        7.0095917 ,  0.18251233,  0.        ,  0.        , -0.        ,
        1.25453722,  0.        , -0.        ,  0.        ,  0.        ,
       -0.        , -0.        , -1.25929175,  0.        , -0.        ,
        1.21821733,  0.        , -0.        ,  0.        ,  4.33177976,
        0.        ,  1.01898197, -0.        ,  0.        ,  0.        ,
        0.        ,  1.4468298 ,  0.46671615, -0.        , -1.5852426 ,
       -3.23725919])

In [29]:
lasso_coefs = pd.Series(lasso_coefs, X.columns)

In [30]:
lasso_coefs[lasso_coefs != 0].sort_values(ascending = False)

Yds            7.009592
Rec            5.335966
Rec_-1_year    4.331780
Yds_-2_year    1.446830
YAC            1.254537
AllPro         1.218217
TD_-1_year     1.018982
TD_-2_year     0.466716
TD             0.182512
Int           -1.259292
Pick          -1.585243
YrsPlayed     -3.237259
dtype: float64

One Standard Deviation increase in last year's receptions will increase target receptions projections by 5.34

## ElasticNet

Because I ran both a Ridge and a Lasso model, I wanted to also run an ElasticNet, which combines the previous two models to hopefully form a better model. 

In [31]:
enet_alphas = np.linspace(0.5, 1.0, 100)
enet_ratio = 0.5
enet_model = ElasticNetCV(alphas = enet_alphas, l1_ratio = enet_ratio, cv=5, max_iter = 100000)
enet_model = enet_model.fit(X_train, y_train)

print(f'Training Score: {enet_model.score(X_train, y_train)}')
print(f'Testing Score: {enet_model.score(X_test, y_test)}')

Training Score: 0.5345167702548312
Testing Score: 0.4975576461287853


Afterwards, I tried adding Polynomial features to both the Ridge, Lasso, and ElasticNet models, but it either made them worse, or threw a bunch of conversion errors, so I decided against using them.

## Principal Component Analysis

In [32]:
X = df.drop(columns = ['Tm', 'Pos', 'Player-additional', 'Tgt_target', 'Rec_target', 'Yds_target', 'TD_target', 'Year'])
y = df['Rec_target']

In [33]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 7)

In [34]:
ss = StandardScaler()
X_train_sc = ss.fit_transform(X_train)
X_test_sc = ss.transform(X_test)

In [51]:
pca = PCA(n_components = 21, random_state = 7)

X_train_pca = pca.fit_transform(X_train_sc)
X_test_pca = pca.transform(X_test_sc)

lr2 = LinearRegression()

lr2.fit(X_train_pca, y_train)

var_exp = pca.explained_variance_ratio_
print(f'Explained variance (first 20 components): {np.round(var_exp[:20], 3)}')
cum_var_exp = np.cumsum(var_exp)
print(f'Cumulative explained variance (first 20 components): {np.round(cum_var_exp[:20], 3)}')

print(f'Training Score: {round(lr2.score(X_train_pca, y_train),4)}')
print(f'Testing Score: {round(lr2.score(X_test_pca, y_test),4)}')

Explained variance (first 20 components): [0.419 0.084 0.081 0.047 0.045 0.043 0.037 0.033 0.028 0.025 0.022 0.017
 0.016 0.014 0.013 0.011 0.01  0.008 0.008 0.007]
Cumulative explained variance (first 20 components): [0.419 0.503 0.584 0.631 0.676 0.719 0.756 0.788 0.816 0.841 0.863 0.88
 0.896 0.911 0.924 0.935 0.945 0.953 0.961 0.968]
Training Score: 0.5315
Testing Score: 0.4957


When running the PCA model, I was playing around with the number of components to use, and ended up finding my best result using the first 21 components.

## Ensemble Model

Now that we've tried a number of models, I will put some of our better scoring models together in a stacking model and see if we can get an even better score.

In [52]:
level1_models = [
    ('rf', RandomForestRegressor()),
    ('gb', GradientBoostingRegressor()),
    ('ada', AdaBoostRegressor()),
    ('lasso_pipe', Pipeline([
        ('ss', StandardScaler()),
        ('lasso', LassoCV())
    ]))
]

stack = StackingRegressor(estimators = level1_models,
                          final_estimator = LinearRegression())

In [53]:
cross_val_score(stack, X_train, y_train).mean()

0.4982010565513463

In [54]:
stack.fit(X_train, y_train)

StackingRegressor(estimators=[('rf', RandomForestRegressor()),
                              ('gb', GradientBoostingRegressor()),
                              ('ada', AdaBoostRegressor()),
                              ('lasso_pipe',
                               Pipeline(steps=[('ss', StandardScaler()),
                                               ('lasso', LassoCV())]))],
                  final_estimator=LinearRegression())

In [55]:
print(f'Training Score: {stack.score(X_train, y_train)}')
print(f'Testing Score: {stack.score(X_test, y_test)}')

Training Score: 0.6348259171201311
Testing Score: 0.49398172060812073


Our best model out of all of the above models ended up being the Gridsearched RandomForest, so we will use that model for making our predictions for receptions.